In [26]:
import pandas as pd

DataFrame 'HECHOS'

In [27]:
df1 = pd.read_excel('Datasets/homicidios.xlsx')
df1.head(1)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


DataFrame 'VICTIMAS'

In [28]:
df2 = pd.read_excel('Datasets/homicidios.xlsx', sheet_name= 'VICTIMAS')
df2.head(1)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00


Se realiza un merge de ambos Datasets para tener centralizada la informacion

In [29]:
df = pd.merge(df1, df2, left_on='ID', right_on='ID_hecho', how= 'inner')
df.set_index('ID', inplace=True)
pd.set_option('display.max_columns', None)
df.head(1)

,N_VICTIMAS,FECHA_x,AAAA_x,MM_x,DD_x,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,ID_hecho,FECHA_y,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00


Se borran columnas innecesarias

In [30]:
df.drop(columns= {'XY (CABA)', 'FECHA_y', 'Dirección Normalizada', 'AAAA_y', 'MM_y', 'DD_y', 'VICTIMA_y', 'FECHA_FALLECIMIENTO'}, inplace=True)

Se eliminan columnas que no son necesarias

In [31]:
nombres = {'N_VICTIMAS':'cant_victimas', 'FECHA_x':'fecha', 'AAAA_x':'año',
                  'MM_x':'mes', 'DD_x':'dia', 'HORA':'hora', 'HH':'franja_horaria', 'LUGAR_DEL_HECHO':'lugar', 'TIPO_DE_CALLE':'tipo_de_calle',
                  'Calle':'calle', 'Altura':'altura', 'Cruce':'cruce','COMUNA':'comuna', 'pos x':'longitud',
                  'pos y':'latitud', 'PARTICIPANTES':'participantes', 'VICTIMA_x':'victima',
                  'ACUSADO':'acusado', 'ROL':'rol', 'SEXO':'sexo', 'EDAD':'edad'}
df = df.rename(columns=nombres)
df.rename_axis('id_hecho', axis='index', inplace=True)
df.drop(columns='ID_hecho', inplace=True)
df.head(1)

,cant_victimas,fecha,año,mes,dia,hora,franja_horaria,lugar,tipo_de_calle,calle,altura,cruce,comuna,longitud,latitud,participantes,victima,acusado,rol,sexo,edad
id_hecho,,,,,,,,,,,,,,,,,,,,,
2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.",8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,CONDUCTOR,MASCULINO,19


Se verifican los tipos de datos de cada clase, para su postrior correcion

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 717 entries, 2016-0001 to 2021-0097
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   cant_victimas   717 non-null    int64         
 1   fecha           717 non-null    datetime64[ns]
 2   año             717 non-null    int64         
 3   mes             717 non-null    int64         
 4   dia             717 non-null    int64         
 5   hora            717 non-null    object        
 6   franja_horaria  717 non-null    object        
 7   lugar           717 non-null    object        
 8   tipo_de_calle   717 non-null    object        
 9   calle           716 non-null    object        
 10  altura          133 non-null    float64       
 11  cruce           540 non-null    object        
 12  comuna          717 non-null    int64         
 13  longitud        717 non-null    object        
 14  latitud         717 non-null    object        
 1

Se verifican columnas de longitud y latitud con solo '.' en sus registros

In [33]:
df4 = df[(df['latitud'] == '.') & (df['longitud'] == '.')]
len(df4)

13

Se reemplaza los registros que tienen punto por nulos

In [34]:
def reemplazar_punto(fila):
    if fila== '.':
        return None
    else:
        return float(fila)
df['latitud'] = df['latitud'].apply(reemplazar_punto)
df['longitud'] = df['longitud'].apply(reemplazar_punto)

Se verifican valores 'SD' en la columna franja horaria

In [35]:
df5 = df[df['franja_horaria']=='SD']
len(df5)

1

Se calcula la moda de la franja horaria

In [36]:
df5 = df[df['franja_horaria']!='SD']
moda = df5['franja_horaria'].mode()
moda

0    7
Name: franja_horaria, dtype: object

se sustituyen los campos 'SD' por la moda del conjunto de datos

In [37]:
def cambiar_int(franja_horaria):
    if franja_horaria == 'SD':
        return moda[0]
    else:
        franja_horaria = int(franja_horaria)
        return franja_horaria
df['franja_horaria'] = df['franja_horaria'].apply(cambiar_int)

Se verifican valores 'SD' en la columna hora

In [38]:
len(df[df['hora']=='SD'])

1

In [39]:
def cambiar_tipo(hora):
    if hora == 'SD':
        return None
    else:
       return hora
df['hora'] = df['hora'].apply(cambiar_tipo)


In [40]:

df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S').dt.time


Se adiciona una columna relacionado la fecha al dia de la semana

In [41]:
df['dia_de_la_semana'] = df['fecha'].dt.day_name()
column_to_move = 'dia_de_la_semana'
new_position = 5  # Puedes ajustar este índice según tu preferencia
current_columns = df.columns.tolist()
current_columns.remove(column_to_move)
current_columns.insert(new_position, column_to_move)
df = df[current_columns]

In [42]:
df.head(1)

,cant_victimas,fecha,año,mes,dia,dia_de_la_semana,hora,franja_horaria,lugar,tipo_de_calle,calle,altura,cruce,comuna,longitud,latitud,participantes,victima,acusado,rol,sexo,edad
id_hecho,,,,,,,,,,,,,,,,,,,,,,
2016-0001,1,2016-01-01,2016,1,1,Friday,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.",8,-58.47534,-34.68757,MOTO-AUTO,MOTO,AUTO,CONDUCTOR,MASCULINO,19


Se verifican valores 'SD' en la columna hora

In [43]:
len(df[df['edad']=='SD'])

53

Se sustituyen los valores 'SD' por la mediana del conjunto de datos

In [44]:
df_mediana = df[df['edad'] != 'SD']
mediana = df_mediana['edad'].median()
df_mediana.shape

(664, 22)

In [45]:
def convertir_edad_to_int(registro):
    if registro == 'SD':
        return int(mediana)
    else:
        return int(registro)
df['edad'] = df['edad'].apply(convertir_edad_to_int)

In [46]:
len(df[df['edad']=='SD'])

0

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 717 entries, 2016-0001 to 2021-0097
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cant_victimas     717 non-null    int64         
 1   fecha             717 non-null    datetime64[ns]
 2   año               717 non-null    int64         
 3   mes               717 non-null    int64         
 4   dia               717 non-null    int64         
 5   dia_de_la_semana  717 non-null    object        
 6   hora              716 non-null    object        
 7   franja_horaria    717 non-null    int64         
 8   lugar             717 non-null    object        
 9   tipo_de_calle     717 non-null    object        
 10  calle             716 non-null    object        
 11  altura            133 non-null    float64       
 12  cruce             540 non-null    object        
 13  comuna            717 non-null    int64         
 14  longitud         

In [48]:
df.to_excel('E:/HENRY_Labs/Data_Analytics/Datasets/data.xlsx')